In [1]:
import re
import pandas as pd
import numpy as np

from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, SmallInteger, BigInteger, Float, String, Date, Time, DateTime 

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)


def connect_db(dbname, host, port, user, password):
    conn_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
    db = create_engine(conn_string)
    return db.connect()

def change_col_names(s: str) -> str:
    s = s.strip()
    s = ''.join([' '+s[i] if s[i].isupper() and (s[i-1].islower() if i>0 else False) else s[i] for i in range(len(s))]).lstrip(' ')
    s = s.capitalize()
    s = re.sub('[^a-zA-Z0-9]', '_', s)
    return ''.join(['_'+i.lower() if i.isupper() else i for i in s]).lstrip('_')

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

HOST = 'camara-dos-deputados.cjdcxj8gvuxr.us-east-2.rds.amazonaws.com'
DB = 'camara_deputados'
LOGIN = os.environ['POSTGRES_USER']
PASSWORD = os.environ['POSTGRES_PASS']

db_connection = connect_db(
        dbname=DB,
        host=HOST,
        port="5432",
        user=LOGIN,
        password=PASSWORD
)        

In [8]:
df_proposicoes = pd.read_csv(Path("data/extrator_files/proposicoes.csv"), sep=';')
df_proposicoes.columns = [change_col_names(i) for i in df_proposicoes.columns]
proposicoes_cols = [
    'id',
    'sigla_tipo',
    'descricao_tipo',
    'ano',
    'ementa',
    'ementa_detalhada',
    'keywords',
    'data_apresentacao',
    'uri_prop_principal',
    'url_inteiro_teor',
    'ultimo_status_data_hora',
    'ultimo_status_sequencia',
    'ultimo_status_id_orgao',
    'ultimo_status_regime',
    'ultimo_status_descricao_tramitacao',
    'ultimo_status_descricao_situacao',
    'ultimo_status_despacho',
    'ultimo_status_url'
]
df_proposicoes = df_proposicoes[proposicoes_cols]
df_proposicoes['id_proposicao_principal'] = df_proposicoes['uri_prop_principal'].str.split('/').str[-1]
df_proposicoes['datetime_apresentacao'] = pd.to_datetime(df_proposicoes['data_apresentacao'], format='%Y-%m-%dT%H:%M:%S')
df_proposicoes['ultimo_status_datetime'] = pd.to_datetime(df_proposicoes['ultimo_status_data_hora'], format='%Y-%m-%dT%H:%M:%S')
df_proposicoes.rename(columns={"id": "id_proposicao", "ano": "ano_apresentacao"}, inplace=True)

# Pegando os temas da proposicoes
df_proposicoes_temas = pd.read_csv(Path("data/extrator_files/proposicoesTemas.csv"), sep=';')
df_proposicoes_temas.columns = [change_col_names(i) for i in df_proposicoes_temas.columns]
df_proposicoes_temas.insert(0, 'id_proposicao', df_proposicoes_temas['uri_proposicao'].str.split('/').str[-1])
df_proposicoes_temas['id_proposicao'] = df_proposicoes_temas['id_proposicao'].astype('Int64')
df_proposicoes_temas = df_proposicoes_temas[['id_proposicao', 'cod_tema', 'tema']]

# Juntados deputados com dados dos temas
df_proposicoes = df_proposicoes.merge(df_proposicoes_temas, on='id_proposicao', how='left')

# Pegando os dados dos autores das proposicoes
df_proposicoes_autores = pd.read_csv(Path("data/extrator_files/proposicoesAutores.csv"), sep=';')
df_proposicoes_autores.columns = [change_col_names(i) for i in df_proposicoes_autores.columns]
df_proposicoes_autores['id_deputado_autor'] = df_proposicoes_autores['id_deputado_autor']
df_proposicoes_autores = df_proposicoes_autores[['id_proposicao', 'id_deputado_autor', 'tipo_autor', 'nome_autor']]
df_proposicoes_autores

# Juntados deputados con autores
df_proposicoes = df_proposicoes.merge(df_proposicoes_autores, on='id_proposicao', how='left')

# Salvando dados
dtypes_proposicoes = {
    'id_proposicao':Integer(),
    'sigla_tipo':String(),
    'descricao_tipo':String(),
    'ementa':String(),
    'ementa_detalhada':String(),
    'cod_tema':Integer(),
    'tema':String(),
    'keywords':String(),
    'id_deputado_autor':Integer(),
    'tipo_autor':String(),
    'nome_autor':String(),
    'datetime_apresentacao':DateTime(),
    'ano_apresentacao':Integer(),
    'id_proposicao_principal':Integer(),
    'url_inteiro_teor':String(),
    'ultimo_status_datetime':DateTime(),
    'ultimo_status_sequencia':Integer(),
    'ultimo_status_id_orgao':Integer(),
    'ultimo_status_regime':String(),
    'ultimo_status_descricao_tramitacao':String(),
    'ultimo_status_descricao_situacao':String(),
    'ultimo_status_despacho':String(),
    'ultimo_status_url':String(),
}
df_proposicoes= df_proposicoes[dtypes_proposicoes.keys()]

nome_da_tabela = 'proposicoes'
df_proposicoes.to_sql(con =db_connection,
                    name=nome_da_tabela,
                    if_exists='replace',
                    index=False, 
                    method='multi',
                    dtype=dtypes_proposicoes
)

C:\Users\Edson Junior\AppData\Local\Temp\ipykernel_18488\3820839713.py:1: DtypeWarning: Columns (8,9,14,19,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_proposicoes = pd.read_csv(Path("data/extrator_files/proposicoes.csv"), sep=';')
C:\Users\Edson Junior\AppData\Local\Temp\ipykernel_18488\3820839713.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_proposicoes['id_proposicao_principal'] = df_proposicoes['uri_prop_principal'].str.split('/').str[-1]
C:\Users\Edson Junior\AppData\Local\Temp\ipykernel_18488\3820839713.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: